# Artificial Neural Network Classification with Python


Libraries import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## The Data

Let's start by reading in the Churn-Modelling.csv file into a pandas dataframe.

In [2]:
train = pd.read_csv('Churn_Modelling.csv')

In [3]:
train.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Converting Categorical Features 

We'll need to convert categorical features to dummy variables using pandas! Otherwise our machine learning algorithm won't be able to directly take in those features as inputs.

In [4]:
Male=pd.get_dummies(train['Gender'],drop_first=True)

In [5]:
Country=pd.get_dummies(train['Geography'],drop_first=True)

In [6]:
train.drop(['RowNumber','CustomerId','Surname','Geography', 'Gender'],axis=1,inplace=True)

In [7]:
train = pd.concat([train,Male,Country],axis=1)

In [8]:
train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [9]:
train.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700,0.545700,0.250900,0.247700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769,0.497932,0.433553,0.431698
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,0.000000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,0.000000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,1.000000,0.000000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,1.000000,1.000000,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,1.000000,1.000000,1.000000


# Data Visualization
Seaborn plot was not supported on this kernal using tensor flow

see backward elimination logistic regression.

In [10]:
view=train[['CreditScore','Age','Balance','EstimatedSalary','Male']]
view.head()

,CreditScore,Age,Balance,EstimatedSalary,Male
0,619,42,0.00,101348.88,0
1,608,41,83807.86,112542.58,0
2,502,42,159660.80,113931.57,0
3,699,39,0.00,93826.63,0
4,850,43,125510.82,79084.10,0


# Building ANN

Let's start by splitting our data into a training set and test set.

## Train Test Split

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('Exited',axis=1), train['Exited'], test_size=0.20, random_state=101)

# Feature Scaling

features must scaled before entering into ANN

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Training and Predicting and Tuning

In [15]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [16]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [17]:
classifier = KerasClassifier(build_fn = build_classifier)

In [18]:
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}

In [19]:
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

In [20]:
grid_search = grid_search.fit(X_train, y_train)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.5471 - acc: 0.7982
Epoch 2/100
7200/7200 [==============================] - 0s 56us/step - loss: 0.4369 - acc: 0.7982
Epoch 3/100
7200/7200 [==============================] - 0s 61us/step - loss: 0.4313 - acc: 0.7982
Epoch 4/100
7200/7200 [==============================] - 1s 77us/step - loss: 0.4274 - acc: 0.7982
Epoch 5/100
7200/7200 [==============================] - 1s 132us/step - loss: 0.4244 - acc: 0.7982
Epoch 6/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.4219 - acc: 0.8040
Epoch 7/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.4204 - acc: 0.8225
Epoch 8/100
7200/7200 [==============================] - 0s 67us/step - loss: 0.4192 - acc: 0.8253
Epoch 9/100
7200/7200 [==============================] - 1s 73us/step - loss: 0.4184 - acc: 0.8249
Epoch 10/100
7200/7200 [===========

7200/7200 [==============================] - 0s 52us/step - loss: 0.4044 - acc: 0.8357
Epoch 80/100
7200/7200 [==============================] - 0s 45us/step - loss: 0.4045 - acc: 0.8354
Epoch 81/100
7200/7200 [==============================] - 0s 45us/step - loss: 0.4044 - acc: 0.8360
Epoch 82/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4042 - acc: 0.8353
Epoch 83/100
7200/7200 [==============================] - 0s 44us/step - loss: 0.4044 - acc: 0.8357
Epoch 84/100
7200/7200 [==============================] - 0s 45us/step - loss: 0.4043 - acc: 0.8349
Epoch 85/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4043 - acc: 0.8342
Epoch 86/100
7200/7200 [==============================] - 0s 48us/step - loss: 0.4038 - acc: 0.8362
Epoch 87/100
7200/7200 [==============================] - 0s 45us/step - loss: 0.4041 - acc: 0.8356
Epoch 88/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4044 - acc: 0.8353
Epoch 89/100


7200/7200 [==============================] - 0s 49us/step - loss: 0.4011 - acc: 0.8354
Epoch 59/100
7200/7200 [==============================] - 0s 45us/step - loss: 0.4008 - acc: 0.8365
Epoch 60/100
7200/7200 [==============================] - 0s 49us/step - loss: 0.4008 - acc: 0.8360
Epoch 61/100
7200/7200 [==============================] - 0s 48us/step - loss: 0.4008 - acc: 0.8369
Epoch 62/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4002 - acc: 0.8368
Epoch 63/100
7200/7200 [==============================] - 0s 48us/step - loss: 0.4001 - acc: 0.8364
Epoch 64/100
7200/7200 [==============================] - 0s 48us/step - loss: 0.4004 - acc: 0.8364
Epoch 65/100
7200/7200 [==============================] - 0s 51us/step - loss: 0.4005 - acc: 0.8364
Epoch 66/100
7200/7200 [==============================] - 0s 46us/step - loss: 0.4002 - acc: 0.8375
Epoch 67/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4003 - acc: 0.8372
Epoch 68/100


7200/7200 [==============================] - 0s 49us/step - loss: 0.4059 - acc: 0.8356
Epoch 38/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4058 - acc: 0.8349
Epoch 39/100
7200/7200 [==============================] - 0s 48us/step - loss: 0.4057 - acc: 0.8350
Epoch 40/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4057 - acc: 0.8346
Epoch 41/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4051 - acc: 0.8364
Epoch 42/100
7200/7200 [==============================] - 0s 44us/step - loss: 0.4057 - acc: 0.8347
Epoch 43/100
7200/7200 [==============================] - 0s 48us/step - loss: 0.4054 - acc: 0.8353
Epoch 44/100
7200/7200 [==============================] - 0s 52us/step - loss: 0.4051 - acc: 0.8361
Epoch 45/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4051 - acc: 0.8349
Epoch 46/100
7200/7200 [==============================] - 0s 50us/step - loss: 0.4050 - acc: 0.8353
Epoch 47/100


7200/7200 [==============================] - 0s 51us/step - loss: 0.3995 - acc: 0.8401
Epoch 17/100
7200/7200 [==============================] - 0s 44us/step - loss: 0.3993 - acc: 0.8400
Epoch 18/100
7200/7200 [==============================] - 0s 49us/step - loss: 0.3988 - acc: 0.8387
Epoch 19/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.3982 - acc: 0.8400
Epoch 20/100
7200/7200 [==============================] - 0s 51us/step - loss: 0.3977 - acc: 0.8381
Epoch 21/100
7200/7200 [==============================] - 0s 48us/step - loss: 0.3971 - acc: 0.8397
Epoch 22/100
7200/7200 [==============================] - 0s 48us/step - loss: 0.3969 - acc: 0.8400
Epoch 23/100
7200/7200 [==============================] - 0s 50us/step - loss: 0.3966 - acc: 0.8403
Epoch 24/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.3963 - acc: 0.8393
Epoch 25/100
7200/7200 [==============================] - 0s 45us/step - loss: 0.3960 - acc: 0.8396
Epoch 26/100


...

8000/8000 [==============================] - 1s 65us/step - loss: 0.3376 - acc: 0.8622
Epoch 167/500
8000/8000 [==============================] - 0s 58us/step - loss: 0.3375 - acc: 0.8624
Epoch 168/500
8000/8000 [==============================] - 0s 49us/step - loss: 0.3373 - acc: 0.8619
Epoch 169/500
8000/8000 [==============================] - 1s 64us/step - loss: 0.3371 - acc: 0.8616
Epoch 170/500
8000/8000 [==============================] - 1s 64us/step - loss: 0.3368 - acc: 0.8622
Epoch 171/500
8000/8000 [==============================] - 0s 56us/step - loss: 0.3366 - acc: 0.8614
Epoch 172/500
8000/8000 [==============================] - 0s 59us/step - loss: 0.3366 - acc: 0.8620
Epoch 173/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.3363 - acc: 0.8639
Epoch 174/500
8000/8000 [==============================] - 1s 92us/step - loss: 0.3361 - acc: 0.8620
Epoch 175/500
8000/8000 [==============================] - 1s 98us/step - loss: 0.3361 - acc: 0.8613
Epoc

8000/8000 [==============================] - 0s 47us/step - loss: 0.3287 - acc: 0.8654
Epoch 247/500
8000/8000 [==============================] - 1s 72us/step - loss: 0.3289 - acc: 0.8652
Epoch 248/500
8000/8000 [==============================] - 0s 53us/step - loss: 0.3282 - acc: 0.8638
Epoch 249/500
8000/8000 [==============================] - 0s 60us/step - loss: 0.3285 - acc: 0.8654
Epoch 250/500
8000/8000 [==============================] - 0s 59us/step - loss: 0.3283 - acc: 0.8654
Epoch 251/500
8000/8000 [==============================] - 1s 63us/step - loss: 0.3285 - acc: 0.8649
Epoch 252/500
8000/8000 [==============================] - 0s 50us/step - loss: 0.3284 - acc: 0.8664
Epoch 253/500
8000/8000 [==============================] - 0s 60us/step - loss: 0.3280 - acc: 0.8658
Epoch 254/500
8000/8000 [==============================] - 0s 48us/step - loss: 0.3282 - acc: 0.8645
Epoch 255/500
8000/8000 [==============================] - 0s 59us/step - loss: 0.3287 - acc: 0.8642
Epoc

Epoch 327/500
8000/8000 [==============================] - 0s 59us/step - loss: 0.3274 - acc: 0.8652
Epoch 328/500
8000/8000 [==============================] - 0s 51us/step - loss: 0.3266 - acc: 0.8644
Epoch 329/500
8000/8000 [==============================] - 0s 61us/step - loss: 0.3268 - acc: 0.8647
Epoch 330/500
8000/8000 [==============================] - 1s 77us/step - loss: 0.3266 - acc: 0.8661
Epoch 331/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.3267 - acc: 0.8668
Epoch 332/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.3271 - acc: 0.8649
Epoch 333/500
8000/8000 [==============================] - 1s 74us/step - loss: 0.3272 - acc: 0.8669
Epoch 334/500
8000/8000 [==============================] - 1s 77us/step - loss: 0.3266 - acc: 0.8645
Epoch 335/500
8000/8000 [==============================] - 0s 54us/step - loss: 0.3267 - acc: 0.8650
Epoch 336/500
8000/8000 [==============================] - 0s 58us/step - loss: 0.3269 - ac

8000/8000 [==============================] - 0s 61us/step - loss: 0.3263 - acc: 0.8659
Epoch 409/500
8000/8000 [==============================] - 0s 57us/step - loss: 0.3270 - acc: 0.8658
Epoch 410/500
8000/8000 [==============================] - 0s 46us/step - loss: 0.3265 - acc: 0.8663
Epoch 411/500
8000/8000 [==============================] - 0s 61us/step - loss: 0.3262 - acc: 0.8664
Epoch 412/500
8000/8000 [==============================] - 0s 59us/step - loss: 0.3269 - acc: 0.8656
Epoch 413/500
8000/8000 [==============================] - 0s 57us/step - loss: 0.3267 - acc: 0.8655
Epoch 414/500
8000/8000 [==============================] - 0s 57us/step - loss: 0.3264 - acc: 0.8669
Epoch 415/500
8000/8000 [==============================] - 0s 60us/step - loss: 0.3266 - acc: 0.8655
Epoch 416/500
8000/8000 [==============================] - 0s 52us/step - loss: 0.3265 - acc: 0.8651
Epoch 417/500
8000/8000 [==============================] - 0s 57us/step - loss: 0.3266 - acc: 0.8660
Epoc

Epoch 489/500
8000/8000 [==============================] - 1s 91us/step - loss: 0.3266 - acc: 0.8656
Epoch 490/500
8000/8000 [==============================] - 0s 59us/step - loss: 0.3263 - acc: 0.8652
Epoch 491/500
8000/8000 [==============================] - 0s 60us/step - loss: 0.3264 - acc: 0.8658
Epoch 492/500
8000/8000 [==============================] - 0s 51us/step - loss: 0.3265 - acc: 0.8649
Epoch 493/500
8000/8000 [==============================] - 0s 59us/step - loss: 0.3267 - acc: 0.8655
Epoch 494/500
8000/8000 [==============================] - 1s 63us/step - loss: 0.3265 - acc: 0.8652
Epoch 495/500
8000/8000 [==============================] - 0s 62us/step - loss: 0.3267 - acc: 0.8660
Epoch 496/500
8000/8000 [==============================] - 0s 54us/step - loss: 0.3268 - acc: 0.8660
Epoch 497/500
8000/8000 [==============================] - 0s 52us/step - loss: 0.3259 - acc: 0.8673
Epoch 498/500
8000/8000 [==============================] - 0s 62us/step - loss: 0.3264 - ac

## Evaluation

Let's move on to evaluate our model!

In [25]:
grid_search.cv_results_

C:\Users\Steven\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Steven\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Steven\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Steven\Anaconda3\lib\site-packages\sklearn\ut

{'mean_fit_time': array([   85.20147004,    52.29840941,  1642.41893461,   278.27726545,
           50.754388  ,    54.08344274,   285.24078937,   270.56803722]),
 'mean_score_time': array([ 0.10232487,  0.21256878,  0.32972887,  0.43180697,  0.54490376,
         0.68102767,  0.82627904,  0.94610398]),
 'mean_test_score': array([ 0.8405  ,  0.83775 ,  0.843125,  0.85225 ,  0.8485  ,  0.83725 ,
         0.847375,  0.852875]),
 'mean_train_score': array([ 0.84266667,  0.84045833,  0.85066667,  0.85915278,  0.851875  ,
         0.83930556,  0.84997222,  0.857125  ]),
 'param_batch_size': masked_array(data = [25 25 25 25 32 32 32 32],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_epochs': masked_array(data = [100 100 500 500 100 100 500 500],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_optimizer': masked_array(data = ['adam' 'rmsprop' 'adam' 'rmsprop' 'adam' 'rmsprop' 'ad

In [23]:
best_parameters = grid_search.best_params_
best_parameters 

{'batch_size': 32, 'epochs': 500, 'optimizer': 'rmsprop'}

In [24]:
best_accuracy = grid_search.best_score_
best_accuracy

0.85287500000000005

# Conclusion
Artificail Nueral Network has an accuracy of 85.3% with 'batch_size': 32, 'epochs': 500, 'optimizer': 'rmsprop'